In [0]:
from pyspark.sql.functions import col, sum, round
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Aggregate the dataset and create various tables
# Aggregate by staff group
gold_SG_df = (spark.table("nhs_silver_workforce")
    .groupBy("PERIOD", "TYPE", "STAFF_GROUP_1_NAME")
    .agg(
        F.sum("FTE").alias("Total_FTE"), 
        F.sum("HC").alias("Total_HC")
    )
    # Force types to prevent the Merge conflict
    .withColumn("Total_FTE", F.col("Total_FTE").cast("double"))
    .withColumn("Total_HC", F.col("Total_HC").cast("long")) 
    .filter(F.col("TYPE").isin("Joiners", "Leavers"))
)
   

# Pivot the dataset
pivot_df = (spark.table("nhs_silver_workforce")
    # Matches Silver column names
    .groupBy(
        "PERIOD", "REPORTING_ORG_CODE", "REPORTING_ORG_NAME", 
        "NHSE_REGION_CODE", "NHSE_REGION_NAME", "ICS_CODE", "ICS_NAME", 
        "CLUSTER_GROUP", "BENCHMARK_GROUP", "STAFF_GROUP_1_NAME"
    )
    #  Pivot on the 'TYPE' column (values must match the data exactly: 'Joiners', 'Leavers', etc.)
    .pivot("TYPE", ["Joiners", "Leavers"]) 
    # 3. Aggregate both metrics
    .agg(
        F.sum("HC").alias("HC"), 
        F.sum("FTE").alias("FTE")
    )
)

# Creating the Regional Benchmarking
gold_regional_df = (spark.table("nhs_silver_workforce")
    .groupBy("PERIOD", "NHSE_REGION_NAME", "STAFF_GROUP_1_NAME")
    .agg(
        # Convert strings to numeric during aggregation
        sum(col("HC").cast("integer")).alias("TOTAL_HC"),
        sum(col("FTE").cast("double")).alias("TOTAL_FTE")
    )
)



# Save to Gold layer
#display(gold_SG_df)
gold_SG_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("nhs_gold_StaffGroup_summary")
pivot_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("nhs_gold_summary")
gold_regional_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold_regional_summary")

